# 🚀 LSTM Christian Text Generator Training

This notebook trains your LSTM model on Google Colab with FREE GPU acceleration.

**Instructions:**
1. Go to Runtime → Change runtime type → Select **T4 GPU**
2. Run all cells in order
3. Training will take ~2-3 hours
4. Model will be saved to your Google Drive

In [ ]:
# 1️⃣ Check GPU availability
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))
!nvidia-smi

In [ ]:
# 2️⃣ Mount Google Drive to save model
from google.colab import drive
drive.mount('/content/drive')

# Create output directory
import os
output_dir = '/content/drive/MyDrive/everyday-christian-model'
os.makedirs(output_dir, exist_ok=True)
print(f"Model will be saved to: {output_dir}")

In [ ]:
# 3️⃣ Upload your training data file
from google.colab import files
import shutil

print("📤 Please upload your training data file (lstm_training_data.txt)")
print("   File location on your Mac:")
print("   /Users/kcdacre8tor/everyday-christian/assets/training_data/lstm_training_data.txt")
print("")
print("Click the 'Choose Files' button that appears below:")

uploaded = files.upload()

# Move uploaded file to expected location
for filename in uploaded.keys():
    shutil.move(filename, '/content/training_data.txt')
    file_size = len(uploaded[filename]) / (1024 * 1024)  # Convert to MB
    print(f"✅ Uploaded {filename} ({file_size:.2f} MB)")

!ls -lh /content/training_data.txt
print("✅ Training data ready!")

In [ ]:
# 4️⃣ Training configuration and model code
import numpy as np
import time

# PROPER Configuration for good results
SEQ_LENGTH = 100
BATCH_SIZE = 256  # GPU can handle larger batches
BUFFER_SIZE = 10000
EMBEDDING_DIM = 256
RNN_UNITS = 1024  # Full size for GPU
EPOCHS = 40  # Proper training (not 5!)
LEARNING_RATE = 0.001

print("📋 Training Configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  RNN Units: {RNN_UNITS}")
print(f"  Learning Rate: {LEARNING_RATE}")

In [ ]:
# 5️⃣ Load and prepare data
def prepare_data():
    print("📖 Loading training data...")
    with open('/content/training_data.txt', 'r', encoding='utf-8') as f:
        text = f.read()
    
    print(f"✅ Loaded {len(text):,} characters")
    
    # Create character mappings
    vocab = sorted(set(text))
    vocab = ['<PAD>', '<START>', '<END>'] + vocab
    
    char2idx = {char: idx for idx, char in enumerate(vocab)}
    idx2char = {idx: char for idx, char in enumerate(vocab)}
    
    print(f"📝 Vocabulary size: {len(vocab)}")
    
    # Save vocabulary
    vocab_path = f'{output_dir}/char_vocab.txt'
    with open(vocab_path, 'w', encoding='utf-8') as f:
        for char in vocab:
            f.write(f"{char}\n")
    
    # Create sequences
    text_as_int = np.array([char2idx.get(c, char2idx[' ']) for c in text])
    char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
    sequences = char_dataset.batch(SEQ_LENGTH + 1, drop_remainder=True)
    
    def split_input_target(chunk):
        input_text = chunk[:-1]
        target_text = chunk[1:]
        return input_text, target_text
    
    dataset = sequences.map(split_input_target)
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
    dataset = dataset.cache().prefetch(tf.data.AUTOTUNE)
    
    return dataset, len(vocab), char2idx, idx2char

# Prepare the data
dataset, vocab_size, char2idx, idx2char = prepare_data()

In [ ]:
# 6️⃣ Build the LSTM model
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(
            vocab_size,
            embedding_dim,
            batch_input_shape=[batch_size, None]
        ),
        tf.keras.layers.LSTM(
            rnn_units,
            return_sequences=True,
            stateful=False,
            recurrent_initializer='glorot_uniform',
            recurrent_dropout=0.1
        ),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.LSTM(
            rnn_units // 2,
            return_sequences=True,
            stateful=False,
            recurrent_initializer='glorot_uniform',
            recurrent_dropout=0.1
        ),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(
            vocab_size,
            kernel_regularizer=tf.keras.regularizers.l2(0.01)
        )
    ])
    return model

# Build model
print("🏗️ Building LSTM model...")
model = build_model(vocab_size, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)

# Compile
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

print(model.summary())

In [ ]:
# 7️⃣ TRAIN THE MODEL - This will take 2-3 hours
print("🎓 Starting training...")
print("⏱️ This will take approximately 2-3 hours with T4 GPU")
print("☕ Go grab a coffee! Colab will keep running.\n")

# Callbacks
checkpoint_path = f"{output_dir}/checkpoint-{{epoch:02d}}"
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        save_weights_only=True,
        save_freq='epoch',
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=5,
        min_delta=0.001,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='loss',
        factor=0.5,
        patience=3,
        min_lr=1e-5,
        verbose=1
    )
]

# Train!
start_time = time.time()
history = model.fit(
    dataset,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

training_time = (time.time() - start_time) / 3600
print(f"\n✅ Training complete in {training_time:.2f} hours!")
print(f"Final Loss: {history.history['loss'][-1]:.4f}")
print(f"Final Accuracy: {history.history['accuracy'][-1]:.4f}")

In [ ]:
# 8️⃣ Rebuild model for inference and convert to TFLite
print("🔄 Rebuilding model for inference...")
inference_model = build_model(vocab_size, EMBEDDING_DIM, RNN_UNITS, batch_size=1)
inference_model.set_weights(model.get_weights())
inference_model.build(tf.TensorShape([1, None]))

# Save Keras model
keras_path = f'{output_dir}/text_generator.h5'
inference_model.save(keras_path)
print(f"✅ Saved Keras model to Google Drive")

# Convert to TFLite (iOS compatible)
print("📦 Converting to TFLite...")
converter = tf.lite.TFLiteConverter.from_keras_model(inference_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# iOS compatible - CPU only operations
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS
]
tflite_model = converter.convert()

# Save TFLite model
tflite_path = f'{output_dir}/text_generator.tflite'
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

print(f"✅ Saved TFLite model to Google Drive")
print(f"📊 Model size: {len(tflite_model) / 1024 / 1024:.2f} MB")

In [ ]:
# 9️⃣ Test the model
def generate_text(model, start_string, char2idx, idx2char, temperature=0.3):
    num_generate = 200
    input_eval = [char2idx.get(s, char2idx.get(' ', 0)) for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    model.reset_states()
    
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    
    return start_string + ''.join(text_generated)

# Test generation
print("🧪 Testing text generation...\n")
test_prompts = ["God ", "I feel ", "When I pray "]

for prompt in test_prompts:
    generated = generate_text(inference_model, prompt, char2idx, idx2char, temperature=0.3)
    print(f"Prompt: '{prompt}'")
    print(f"Generated: {generated[:150]}...\n")

In [ ]:
# 🎉 DOWNLOAD YOUR MODEL
print("✨ Training Complete!\n")
print("📦 Your trained models are saved in Google Drive:")
print(f"   TFLite: {tflite_path}")
print(f"   Vocabulary: {output_dir}/char_vocab.txt\n")
print("To download to your computer:")
print("1. Go to Google Drive → everyday-christian-model folder")
print("2. Download text_generator.tflite and char_vocab.txt")
print("3. Copy them to your Flutter app's assets/models/ folder\n")
print("Or download directly from Colab:")

from google.colab import files
print("\n🔽 Downloading model files to your computer...")
files.download(tflite_path)
files.download(f'{output_dir}/char_vocab.txt')